# Preprocessing training data

This is the continuation of my previous article. In this article, I will apply a preprocessing method on the dataset that was presented in the previous article (*Keras1.ipynb*), followed by the training of the deep learning networks. At the end, testing results will be presented.

It should be noted that the dataset that was presented in the previous article had uneven distribution of true and false cases. In particular, the dataset had 268 true cases and 500 false cases of diabetese. Details on the dataset can be found in the following link: 
https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names

The deep learning model trained using such dataset can lead to the biasness in the trained network outcome because more true cases are fed to the network than false cases. Such issues has been discussed in the following article by Satya Mallick:

https://www.learnopencv.com/bias-variance-tradeoff-in-machine-learning/

To avoid it, the dataset should have equal numbers of true and false cases, respectively. Therefore let us create new data that has the property. Furthermore, the training and testing of the model should be done in different sets. Therefore, the data set will be divided into 4 such sets, and one set will be used to train, whereas the remainings are used for testing the model.

The preprocessing steps can be explained by the following figure
![alt text]()